# Download Data with `earthaccess`

## STEP 1: Set up

Load your stored variables into memory:

In [1]:
%store -r denv_redlin_gdf data_dir ndvi_dat ndbi_dat

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>Add imports for packages that help you:</p>
<ol type="1">
<li>Work with the file system interoperably</li>
<li>Work with vector data</li>
<li>Create interactive plots of vector data</li>
<li>Group and aggregate tabular data</li>
</ol></div></div>

In [2]:
import re # Use regular expressions to extract metadata

import earthaccess # Access NASA data from the cloud
# Overlay raster and vector data
import numpy as np # Process bit-wise cloud mask
# Group and aggregate
import pandas as pd
#plot
import matplotlib.pyplot as plt
# Work with raster data
import rioxarray as rxr
from rioxarray.merge import merge_arrays # Merge rasters

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Set up `earthaccess` connection</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Make an account on the <a
href="https://urs.earthdata.nasa.gov/">earthdata site</a>. You don’t
need to spend a lot of time on the registration form – go ahead answer
their questions to the best of your ability. That information is used
for internal reporting and analysis, not to decide whether or not to
grant you an account!</li>
<li>Run the code below an enter your credentials to log into earthaccess
from Python. You should only need to do this once, as long as
<code>persist=True</code> is set.</li>
</ol></div></div>

In [3]:
earthaccess.login(strategy="interactive", persist=True)

## STEP 2: Search

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Search for HLS data</div></div><div class="callout-body-container callout-body"><p>It can be useful to use NASA’s online resources to find the data you
want to download before accessing it over API. To do that:</p>
<ol type="1">
<li>Go to the <a href="https://worldview.earthdata.nasa.gov/">NASA
Worldview Site</a> and search for Denver, CO.</li>
<li>Add the <code>HLSL30</code> product as a base map. You can do this
by clicking <code>Add Layer</code> and then searching for the
<strong>short name</strong> <code>HLSL30</code>. When looking for other
datasets, the short name can usually be found by on the data homepage,
accessible from the <strong>doi</strong>. Practice opening the data page
for <code>HLSL30</code> to find where the shortcode is.</li>
<li>By dragging the date indicator on the bottom, search the month of
July, 2023 for a day that has data available and little to no cloud
cover over Denver. You may need to wait a second or so for the data to
load for any given date.</li>
</ol>
<p>Now, using the code below:</p>
<ol type="1">
<li>Put the short name of the dataset into the
<code>earthaccess.search_data()</code> function.</li>
<li>Replace <code>gdf</code> with the name of your denver redlining
<code>GeoDataFrame</code>.</li>
<li>Put the date you found into the <code>temporal</code> parameter,
replacing <code>YYYY-MM-DD</code> with the appropriate year (Y), month
(M), and day (D) digits.</li>
<li>Run the code and make sure there are results! If not, double check
your date.</li>
</ol></div></div>

In [4]:
denver_results = earthaccess.search_data(
    short_name="HLSL30",
    bounding_box=tuple(denv_redlin_gdf.total_bounds),
    temporal=("2023-07-12", "2023-07-12"),
    count=1
)
denver_results

[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -105.84822916, 'Latitude': 38.7577866}, {'Longitude': -104.88766444, 'Latitude': 38.76081048}, {'Longitude': -104.88607437, 'Latitude': 39.75021202}, {'Longitude': -105.55700976, 'Latitude': 39.74893151}, {'Longitude': -105.84822916, 'Latitude': 38.7577866}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2023-07-12T17:36:53.600Z', 'EndingDateTime': '2023-07-12T17:37:17.491Z'}}
 Size(MB): 132.62681674957275
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.SZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.B04.tif', 'https://dat

## STEP 3: Open data connections

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Access HLS data</div></div><div class="callout-body-container callout-body"></div></div>

In [5]:
denver_files = earthaccess.open(denver_results)

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
denver_files

[<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.SZA.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.B04.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.B07.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.B01.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T13SDD.2023193T173653.v2.0/HLS.L30.T13SDD.2023193T173653.v2.0.VAA.tif>,
 <File-like object HTTPFileSystem, https://data.lpdaac.earth